---
## <font color=#FF8181>Unit 9 - Exercises: </font>

In the following exercises, we will leverage knowledge from previous units and test our understanding on __Time Series__. We shall be using YahooFinancials to conduct Time Series Analysis on historical stock prices. Refer to the materials on Time Series and Aggregations for guidance on how to obtain data from YahooFinancials.

In [1]:
import pandas as pd
import numpy as np
from yahoofinancials import YahooFinancials

---
### <font color=#14F278> Task 1:  </font>

<font color=#14F278> **The purpose of Task 1 is to produce a Time Series of average yearly Apple stock prices for the years 2019-2021 via downsampling and present it in a pre-defined format. This exercise tests your knowledge on Time Series, as well as melting and sorting a dataframe. The task deliverable is to produce the dataframe below:** </font>

<center>
    <div>
        <img src="..\images\timeseries_003.png"/>
    </div>
</center>


__Steps: Create a python function `apple_downsample()` which does the following:__
- Scrapes historical monthly price data on Apple from __YahooFinancials__
- Constructs a DataFrame with the monthly price observations for period 2019 - 2021 - columns 'high', 'low' and 'close'
- Sets column 'formatted_date' to be your DataFrame index
- Downsamples DataFrame from monthly to annual level of granularity via averaging monthly observations
- Resets index to new DataFrame (to ensure that 'formatted_date' values are in a column)
- Melts the DataFrame, so that column names 'high', 'low', 'close' become entries of column __'price_type'__ and observations are stored in column __'avg_price'__
- Sorts melted DataFrame by 'formatted_date' and 'price_type' in ascending order
- Returns final sorted DataFrame

In [30]:
# Write your solution here
def apple_downsample():
    import yahoofinancials
    from yahoofinancials import YahooFinancials
    data = YahooFinancials('AAPL').get_historical_price_data('2019-01-01', '2021-01-01', 'monthly')
    prices = pd.DataFrame(data['AAPL']['prices'])
    prices.head()
    prices.drop(columns = ['date','volume','adjclose'], inplace = True)
    prices['formatted_date'] = pd.to_datetime(prices['formatted_date'])
    prices.set_index('formatted_date', inplace = True)
    prices_new = prices.resample('Y').mean()
    prices_new.reset_index(inplace = True)
    prices_melt = prices_new.melt(id_vars = ['formatted_date'], var_name = 'price_type',
                    value_name = 'avg_price')
    prices_melt = prices_melt.sort_values(['formatted_date','price_type'])
    return prices_melt

apple_downsample()

C:\Users\MaxBeck\AppData\Local\Temp\ipykernel_29472\3712514371.py:11: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  prices_new = prices.resample('Y').mean()


,formatted_date,price_type,avg_price
6,2019-12-31,close,53.304375
0,2019-12-31,high,55.113750
2,2019-12-31,low,48.476251
4,2019-12-31,open,50.804791
7,2020-12-31,close,97.094583
1,2020-12-31,high,104.093332
3,2020-12-31,low,86.300833
5,2020-12-31,open,92.777499


---
### <font color=#14F278> Task 2:</font>

<font color=#14F278> **The purpose of Task 2 is to analyse the deivation which linear interpolation produces on a Time Series. We know that linear interpolation effectively 'smoothes out' a Time Series - in this exercise you will be asked to quantify this 'smoothing effect' in the context of Apple daily stock prices for Q4 2021. Task 2 aims to compare the real daily stock prices to linearly interpolated ones and assess the percentage deviation between the two. The deliverable is to produce the dataframe below (NB: image shows a part of the dataframe):** </font>


<center>
    <div>
        <img src="..\images\timeseries_004.png"/>
    </div>
</center>



__Steps: Create a python function `apple_upsample()` which does the following:__
- Scrapes historical monthly price data on Apple from __YahooFinancials__
- Creates two dataframes - __monthly_prices__ and __daily_prices__:
    - __monthly_price__ containing the monthly price observations for Q4 2021 - column 'close'
    - __daily_price__ containing the daily price observations for Q4 2021 - column 'close'
- Sets column 'formatted_date' to be the index to __monthly_prices__ dataframe
- Upsamples __monthly_prices__ to daily level of granularity
- Fills in missing values via Linear Interpolation
- Resets index to __monthly_prices__ (to ensure that 'formatted_date' values are in a column)
- Performs a left merge of __monthly_prices__ onto __daily_prices__
- Renames column __close_x__ to __real_close__ and __close_y__ to __inter_close__
- Creates a new column __'perc_dev'__, calculating the deviation of the interpolated price value from the real price value on a daily basis 
     - __perc dev__ = ((__inter_close__ - __real_close__)/__real_close__)*100)
- Returns the final DataFrame

In [47]:
# Write your solution here #TODO
def apple_upsample():
    import yahoofinancials
    from yahoofinancials import YahooFinancials
    data = YahooFinancials('AAPL').get_historical_price_data('2021-10-01', '2021-12-31', 'monthly')
    data2 = YahooFinancials('AAPL').get_historical_price_data('2021-10-01', '2021-12-31', 'daily')
    prices = pd.DataFrame(data['AAPL']['prices'])
    prices.head()
    prices.drop(columns = ['date','volume','adjclose','high','low','open'], inplace = True)
    prices['formatted_date'] = pd.to_datetime(prices['formatted_date'])
    prices.set_index('formatted_date', inplace = True)
    monthly_prices = prices
    prices = pd.DataFrame(data2['AAPL']['prices'])
    prices.head()
    prices.drop(columns = ['date','volume','adjclose','high','low','open'], inplace = True)
    prices['formatted_date'] = pd.to_datetime(prices['formatted_date'])
    prices.set_index('formatted_date', inplace = True)
    daily_prices = prices

    day_up_prices = monthly_prices.resample('D').mean()
    day_up_prices = day_up_prices.interpolate()
    day_up_prices.reset_index(inplace = True)

    merged = pd.merge(daily_prices, day_up_prices, on = 'formatted_date', how = 'left')

    #TODO
    # bit of an isuse with some of the values being nan for some reason

    return merged

apple_upsample()

,formatted_date,close_x,close_y
0,2021-10-01,142.649994,149.800003
1,2021-10-04,139.139999,151.300003
2,2021-10-05,141.110001,151.800003
3,2021-10-06,142.000000,152.300003
4,2021-10-07,143.289993,152.800003
...,...,...,...
58,2021-12-23,176.279999,NaN
59,2021-12-27,180.330002,NaN
60,2021-12-28,179.289993,NaN
61,2021-12-29,179.380005,NaN


---
### <font color=#14F278> Task 3:</font>

<font color=#14F278> **The purpose of Task 3 is to detect the dates in December 2021 when Apple stock prices increased by more than 2% compared to the previous day. This exercise tests your knowledge on performing rolling statistics on a Time Series, as well as filtering a dataframe via a Boolean Mask. The task deliverable is to produce the dataframe below:**</font>


<center>
    <div>
        <img src="..\images\timeseries_005.png"/>
    </div>
</center>


__Steps: Create a python function `apple_rolling_stats()` which does the following:__
- Scrapes historical monthly price data on Apple from __YahooFinancials__
- Constructs a DataFrame with the daily price observations for December 2021 - column 'close'
- Sets column 'formatted_date' to be your DataFrame index
- Calculates the __percentage change__ of the Time Series across observations - use `.pct_change()` and store in column __perc_change__
- Takes the __first difference__ of the Time Series - use `.diff(1)` and store in column __first_diff__
- Filters the resulting DataFrame via __Boolean Mask__ to contain only daily observations with __percentage change__ above 2% (0.02)
- Resets index to new DataFrame (to ensure that 'formatted_date' values are in a column)
- Returns the final DataFrame

In [ ]:
# Write your solution here

__NB__: *Solutions to these exercises are distributed separately in the form of a stand-alone unit at a later point in time. This is to ensure that consultants have had the chance to attempt the exercises autonomously, leveraging the reading materials and concept check solutions.*